In [1]:
"""
    @author Bharatwaaj Shankaranarayanan
    @application hotel-score-model
    @description This application takes the customer reviews as input and gives a rating to a particular hotel
    @version 1.0.0
"""

# Imports
from google.cloud import language_v1
from flask import escape
# import functions_framework
import uuid
import json
import firebase_admin
from firebase_admin import firestore
import time

firebase_admin.initialize_app()
db = firestore.client()


class HoteScoreModel():
    
    """
        @author Bharatwaaj Shankaranarayanan
        @class HotelScoreModel
        @description HotelScoreModel Generi class
        @params
        @return Object
    """
    
    # Constructor for the HotelScoreModel
    def __init__(self):
        
        # Creating the Language Service Client
        self.client = language_v1.LanguageServiceClient()
        
    def analyze(self, data):
        
        """
            @author Bharatwaaj Shankaranarayanan
            @function analyze
            @description This function analyzes the data and provides the sentiment results from GCP Cloud Natural Language API
            @params data (str)
            @returns annotations (AnalyzeSentimentResponse)
        """
        
        # Create the content document from the set of input sentences
        document = language_v1.Document(content=data,type_=language_v1.Document.Type.PLAIN_TEXT)
        
        # Get the annotations of the document from GCP Natural Language API client
        annotations = self.client.analyze_sentiment(request={"document": document})
        
        return annotations
    
    def generate_rating_score(self, annotations):
        """
            @author Bharatwaaj Shankaranarayanan
            @function generate_rating_score
            @description This function parses the annotations for the data generated by the GCP Natural Language API and generates the score
            @params annotations (AnalyzeSentimentResponse)
            @returns score (float)
        """
        return ((annotations.document_sentiment.score + 1) * 5) / 2
    
    def generate_rating_polarity(self, annotations):
        """
            @author Bharatwaaj Shankaranarayanan
            @function generate_rating_polarity
            @description This function parses the annotations for the data generated by the GCP Natural Language API and generates rating polity
            @params annotations (AnalyzeSentimentResponse)
            @returns polarity (enum)
        """
        if(annotations.document_sentiment.score < 0):
            return "NEGATIVE";
        elif (annotations.document_sentiment.score == 0):
            return "NEUTRAL";
        else:
            return "POSITIVE";
    
def main(request):
    if request.method == 'OPTIONS':
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': '*',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Max-Age': '3600'
        }

        return ('', 204, headers)

    request_json = request.get_json(silent=True)
    request_args = request.args
    path = request.path.split("/")

    headers = {
        'Access-Control-Allow-Origin': '*'
    }

    route = path[-1]
    try:
        if request.method == "GET":
            if route == "":
                print("fetching hotels...")
                return ({"data": get_all_hotels()}, 200, headers)
            elif route == "rooms":
                hotel_id = request_args["hotel_id"]
                start_date = request_args["start_date"]
                end_date = request_args["end_date"]
                return ({"data": get_all_rooms(hotel_id, start_date, end_date)}, 200, headers)
            elif route == "history":
                user_id = request_args["user_id"]
                return ({"data": get_user_booking_history(user_id)}, 200, headers)
            else:
                return ({"data": []}, 404, headers)

        elif request.method == "POST":
            pprint(request_args)
            pprint(request_json)
            hotel_id = request_args["hotel_id"]
            room_id = request_args["room_id"]
            user_id = request_json["user_id"]
            start_date = request_json["start_date"]
            end_date = request_json["end_date"]

            print(f"booking room with following details : hotel_id: {hotel_id},"
                  f"room_id: {room_id}, user_id: {user_id}, start_date: {start_date}, end_date: {end_date}")

            return ({"data": book_hotel_room(hotel_id, room_id, user_id, start_date, end_date)}, 200, headers)
        else:
            return ({"data": []}, 404, headers)

    except Exception as e:
        return ({"message": "Something went wrong! Try Again"}, 500, headers)    
    
    
    
# save_reviews("6c0de08a-4239-4a81-a53e-1e185d98df06", "e3ca46f9-7574-4adc-ab90-b2f1a7a6ed4b", "This is very poor service.")

review_data {'user_id': '6c0de08a-4239-4a81-a53e-1e185d98df06', 'hotel_id': 'e3ca46f9-7574-4adc-ab90-b2f1a7a6ed4b', 'review': 'This is very poor service.', 'score': 0.4999999701976776, 'polarity': 'NEGATIVE'}


In [4]:
import sys

In [5]:
!{sys.executable} -m pip install firebase_admin

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 KB 899.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.5/244.5 KB 726.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 8.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 2.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
save_reviews("6c0de08a-4239-4a81-a53e-1e185d98df06", "e3ca46f9-7574-4adc-ab90-b2f1a7a6ed4b", "Highly average service. Poor food. Overall it was okayish experience.")

review_data {'user_id': '6c0de08a-4239-4a81-a53e-1e185d98df06', 'hotel_id': 'e3ca46f9-7574-4adc-ab90-b2f1a7a6ed4b', 'review': 'Highly average service. Poor food. Overall it was okayish experience.', 'score': 1.7499999701976776, 'polarity': 'NEGATIVE'}


In [4]:
from services import get_rating_by_hotel, save_reviews

In [2]:
get_rating_by_hotel("e3ca46f9-7574-4adc-ab90-b2f1a7a6ed4b")

{'rating': 1.1249999701976776, 'success': True}

In [ ]:
def main(request):
    if request.method == 'OPTIONS':
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': '*',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Max-Age': '3600'
        }

        return ('', 204, headers)

    request_json = request.get_json(silent=True)
    request_args = request.args
    path = request.path.split("/")

    headers = {
        'Access-Control-Allow-Origin': '*'
    }

    route = path[-1]
    try:
        if request.method == "POST":
            
            hotel_id = request_args["hotel_id"]
            
            if route == "save_reviews":
                user_id = request_json["user_id"]
                review = request_json["review"]
                print(f"save_reviews called : hotel_id: {hotel_id}, user_id: {user_id}, review: {review}")
                return ({"data": save_reviews(user_id, hotel_id, review)}, 200, headers)
        
            elif route == "get_rating_by_hotel":
                print(f"save_reviews called : hotel_id: {hotel_id}")
                return ({"data": get_rating_by_hotel(hotel_id)}, 200, headers)
        
        
        else:
            return ({"data": []}, 404, headers)

    except Exception as e:
        return ({"message": "Something went wrong! Try Again"}, 500, headers)   